In [2]:
from rdkit.Chem import DataStructs
from tqdm import tqdm
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from matplotlib import pyplot as plt
import pandas as pd
from moses.metrics import weight, logP, SA, QED,mol_passes_filters
from moses.metrics.utils import get_mol, mapper

from rdkit import Chem
from rdkit.Chem import AllChem,Descriptors,Lipinski,DataStructs
from functools import partial
import numpy as np
from sklearn import manifold
import random
import glob
import os
import seaborn as sns

In [6]:
from iter_gen import task as task_dict
df_dict={}
color_dict={'ReINVENT':'purple','RationaleRL':'green','MARS':'orange','AIxFuse_init':'blue','AIxFuse':'red','rorgt':'black','dhodh':'grey','gsk3b':'black','jnk3':'grey'}
task='gsk3b_jnk3'
target1,target2=task.split('_')
pdb_id1=task_dict[task]['pdb_id1']
pdb_id2=task_dict[task]['pdb_id2']
print(pdb_id1,pdb_id2)
prec=task_dict[task]['prec']
for csv in glob.glob(os.path.join('data/compare',task,'*.csv')):
    name=csv.split('/')[-1].split('.')[0]
    if name in color_dict:
        df_dict[name]=(pd.read_csv(csv),name,color_dict[name])

6Y9S 4WHZ


In [10]:
metric_dict={
    f'{pdb_id1}_{prec}':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse',target1]],
    f'{pdb_id2}_{prec}':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse',target2]],
    f'LogP':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse',target1,target2]],
    f'SA':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse',target1,target2]],
    f'QED':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse',target1,target2]],
    f'Weight':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse',target1,target2]],
}
# metric_dict={
#     f'{pdb_id1}_{prec}':[df_dict[name] for name in ['AIxFuse_init','AIxFuse',target1]],
#     f'{pdb_id2}_{prec}':[df_dict[name] for name in ['AIxFuse_init','AIxFuse',target2]],
#     # f'LogP':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse_init','AIxFuse',target1,target2]],
#     f'SA':[df_dict[name] for name in ['AIxFuse_init','AIxFuse',target1,target2]],
#     f'QED':[df_dict[name] for name in ['AIxFuse_init','AIxFuse',target1,target2]],
#     # f'Weight':[df_dict[name] for name in ['ReINVENT','RationaleRL','MARS','AIxFuse_init','AIxFuse',target1,target2]],
# }
metric_14d={'QED':0.368308769245971,'SA':3.74619658218488,'5NTP_XP':-12.9913537202175,'6QU7_XP':-10.1629030495152,'LogP':6.71892,'Weight':585.998}
for metric in metric_dict:
    fig=plt.figure(figsize=(9,3))
    plt.grid(True,color="gray",linewidth="0.5",axis="both")
    for model,model_name,color in metric_dict[metric]:
        if model_name =='AIxFuse_init':
            model_name = 'AIxFuse(w/o ALRL)'
        col=model[metric].dropna()
        # col[col.isnull()]=0
        sns.distplot(col, hist=False, kde=True,
            kde_kws={'fill': True, 'linewidth': 3},
            label=model_name,color=color)
    plt.ylim(top=plt.ylim()[1])
    if task=='rorgt_dhodh':
        plt.plot([metric_14d[metric],metric_14d[metric]],[0,plt.ylim()[1]],label='JMC (R)-14d')
    plt.xlabel(metric,fontdict={'family' : 'Times New Roman', 'size'   : 20})
    plt.ylabel('Density',fontdict={'family' : 'Times New Roman', 'size'   : 20})
    plt.xticks(fontproperties = 'Times New Roman', size = 14)
    plt.yticks(fontproperties = 'Times New Roman', size = 14)
    # if metric=='QED':
    #     plt.legend(loc='upper left',fontsize=16)
    # else:
    plt.legend(fontsize=14)
    plt.tight_layout()
    # plt.xlim(left=0)
    # plt.ylim(bottom=plt.ylim()[0]+plt.ylim()[1]/100)
    fig.savefig(os.path.join('data/outputs/images/distribution',task,f'{metric}.png'),dpi=250)
    plt.close()